# Build Data for Predictive Modeling

### Load Necessary Packages

In [14]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

### Create function for checking memory usage of dataframes

In [15]:
def check_memory_usage(data):
    """ Takes DataFrame as input and returns memory usage statistics. 
    """
    print ''
    print 'Checking memory usage statistics... '
    print(data.info(memory_usage=True))      

### Load the data

In [16]:
def load_data(): 
    """ Loads and returns competition data, including modified clients and products data.
    """
    # Set work directory
    os.chdir('D:/OneDrive/Documents/Kaggle/Grupo Bimbo Inventory Demand/data')      
    
    # Load data efficiently to use less RAM
    train_types = {
                    'Semana': np.uint8, 'Agencia_ID': np.uint16, 'Canal_ID': np.uint8
                    ,'Ruta_SAK': np.uint16, 'Cliente_ID': np.uint32, 'Producto_ID': np.uint16
                    ,'Demanda_uni_equil': np.uint16
                    }   
         
    test_types =  {
                    'Semana': np.uint8, 'Agencia_ID': np.uint16, 'Canal_ID': np.uint8
                    ,'Ruta_SAK': np.uint16, 'Cliente_ID': np.uint32, 'Producto_ID': np.uint16
                    }
                    
    product_types = {'Producto_ID': np.uint16, 'weight': np.float32, 'pieces': np.float32
                    , 'weight_per_piece': np.float32, 'short_product_name': np.object
                    }
    
    train = pd.read_csv("train.csv", sep = ",", usecols = train_types.keys(), dtype = train_types) 
    test = pd.read_csv("test.csv", sep = ",", usecols = test_types.keys(), dtype = test_types)
    
    clients = pd.read_csv("cliente_tabla_modified.csv", sep = ",") 
    products = pd.read_csv("producto_tabla_modified.csv", sep = ",", usecols = product_types.keys(), dtype = product_types) 
        
    # Return data
    return train, test, clients, products

In [17]:
print "1. Loading data..."
train, test, clients, products = load_data()
print "Complete!"
print ''
check_memory_usage(train)

1. Loading data...
Complete!


Checking memory usage statistics... 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74180464 entries, 0 to 74180463
Data columns (total 7 columns):
Semana               uint8
Agencia_ID           uint16
Canal_ID             uint8
Ruta_SAK             uint16
Cliente_ID           uint32
Producto_ID          uint16
Demanda_uni_equil    uint16
dtypes: uint16(4), uint32(1), uint8(2)
memory usage: 990.4 MB
None


### Merge data

In [18]:
def merge_data(train, test, clients, products):
    """ Merges client and product data with train and test data. 
        Returns modified train and test data.
    """
      
    # Merge product information
    train = train.merge(products, how = 'left', on = ['Producto_ID'])
    test = test.merge(products, how = 'left', on = ['Producto_ID'])

    # Merge cliente information
    train = train.merge(clients.ix[:, ['Cliente_ID', 'Client_Type']], how = 'left', on = ['Cliente_ID'])
    test = test.merge(clients.ix[:, ['Cliente_ID', 'Client_Type']], how = 'left', on = ['Cliente_ID'])         

    return train, test  

In [19]:
print "2. Merging data..."
train, test = merge_data(train, test, clients, products) 
print "Complete!"
print ""
check_memory_usage(train)

2. Merging data...
Complete!


Checking memory usage statistics... 
<class 'pandas.core.frame.DataFrame'>
Int64Index: 74180464 entries, 0 to 74180463
Data columns (total 12 columns):
Semana                uint8
Agencia_ID            uint16
Canal_ID              uint8
Ruta_SAK              uint16
Cliente_ID            uint32
Producto_ID           uint16
Demanda_uni_equil     uint16
weight                float32
pieces                float32
weight_per_piece      float32
short_product_name    object
Client_Type           object
dtypes: float32(3), object(2), uint16(4), uint32(1), uint8(2)
memory usage: 3.5+ GB
None


### Add time series lag variables to data

In [ ]:
def add_time_series_features(train, test):
    """ Takes train and test data as inputs. 
        Adds lagged adjusted demand variables to the data periods t-2, t-3, t-4, and t-5.
        Returns modified data. 
    """
    # Add lagged time series variables to train/test
    # Groups by (Cliente_ID, Producto_ID)
    for i in range(1, 6):
        # Add tminus to train
        columns = ['Semana', 'Cliente_ID', 'Producto_ID', 'Demanda_uni_equil']       
        train_tminus = train.ix[:, columns]
        train_tminus = train_tminus.rename(columns = {'Demanda_uni_equil': 'Demanda_uni_equil_tminus' + str(i)})
        train_tminus['Semana'] = train_tminus['Semana'] + i
        columns.remove('Demanda_uni_equil')
        train_tminus = pd.DataFrame({'Demanda_uni_equil_tminus' + str(i) : train_tminus.groupby(columns)['Demanda_uni_equil_tminus' + str(i)].mean()}).reset_index()         
        
        train = train.merge(train_tminus, how = 'left', on = ['Semana', 'Cliente_ID', 'Producto_ID'])   
        train['Demanda_uni_equil_tminus' + str(i)] = train['Demanda_uni_equil_tminus' + str(i)].astype(np.float16)
                
        # Add tminus for Semana == 10
        columns = ['Semana', 'Cliente_ID', 'Producto_ID', 'Demanda_uni_equil_tminus' + str(i)]
        test = test.merge(train_tminus.ix[train_tminus['Semana'] == 10, columns], how = 'left', on = ['Semana', 'Cliente_ID', 'Producto_ID'])
        test['Demanda_uni_equil_tminus' + str(i)] = test['Demanda_uni_equil_tminus' + str(i)].astype(np.float16)     
    
        # Add tminus for Semana == 11
        if(i != 1):
            test = test.merge(train_tminus.ix[train_tminus['Semana'] == 11, columns], how = 'left', on = ['Semana', 'Cliente_ID', 'Producto_ID']) 
            test = test.rename(columns = {'Demanda_uni_equil_tminus' + str(i) + '_x': 'Demanda_uni_equil_tminus' + str(i)})
            test.ix[test['Semana'] == 11, 'Demanda_uni_equil_tminus' + str(i)] = test.ix[test['Semana'] == 11, 'Demanda_uni_equil_tminus' + str(i) + '_y']
            test = test.drop('Demanda_uni_equil_tminus' + str(i) + '_y', axis = 1)
            test['Demanda_uni_equil_tminus' + str(i)] = test['Demanda_uni_equil_tminus' + str(i)].astype(np.float16)
        
        # Replace null values with zeros
        train.ix[train['Demanda_uni_equil_tminus' + str(i)].isnull(), 'Demanda_uni_equil_tminus' + str(i)] = 0
        test.ix[test['Demanda_uni_equil_tminus' + str(i)].isnull(), 'Demanda_uni_equil_tminus' + str(i)] = 0    
            
    return train, test  

In [ ]:
print "3. Adding time series features..."
train, test = add_time_series_features(train, test)  
print "Complete!"
print ""
check_memory_usage(train)

3. Adding time series features...


### Add mean weekly frequency variables for categorical id variables 

In [20]:
def add_mean_freq_by_id(train, test):
    """ Takes train and test data as inputs. 
        Adds mean of frequencies of different id features by week (semana).
        Returns train and test data.
    """
    columns =  ['Agencia_ID', 'Canal_ID', 'Ruta_SAK', 'Cliente_ID', 'Producto_ID', 'short_product_name', 'Client_Type'] 
    for column in columns:
        
        # Create mean of weekly id counts
        train_counts = pd.DataFrame({column + '_count' : train[[column, 'Semana']].groupby([column, 'Semana']).size()}).reset_index()
        test_counts = pd.DataFrame({column + '_count' : test[[column, 'Semana']].groupby([column, 'Semana']).size()}).reset_index()
        counts = train_counts.append(test_counts)      
        counts = pd.DataFrame({column + '_count' : counts.groupby([column])[column + '_count'].mean()}).reset_index()
        counts[column + '_count'] = counts[column + '_count'].astype(np.float32) 

        # Merge with train and test data
        train = train.merge(counts, how = 'left', on = column)    
        test = test.merge(counts, how = 'left', on = column)  
        
    return train, test

In [22]:
#print train.columns
print "4. Adding id frequency variables... "
train, test = add_mean_freq_by_id(train, test)
print "Complete!"
print ''
check_memory_usage(train)

4. Adding id frequency variables... 
Complete!


Checking memory usage statistics... 
<class 'pandas.core.frame.DataFrame'>
Int64Index: 74180464 entries, 0 to 74180463
Data columns (total 19 columns):
Semana                      uint8
Agencia_ID                  uint16
Canal_ID                    uint8
Ruta_SAK                    uint16
Cliente_ID                  uint32
Producto_ID                 uint16
Demanda_uni_equil           uint16
weight                      float32
pieces                      float32
weight_per_piece            float32
short_product_name          object
Client_Type                 object
Agencia_ID_count            float32
Canal_ID_count              float32
Ruta_SAK_count              float32
Cliente_ID_count            float32
Producto_ID_count           float32
short_product_name_count    float32
Client_Type_count           float32
dtypes: float32(10), object(2), uint16(4), uint32(1), uint8(2)
memory usage: 5.4+ GB
None


### Encode categorical variables 

In [ ]:
def encode_labels(train, test, threshold): 
    """ Converts categorical features to integers in train and test data.
        Groups values in categorical features appearing less than threshold into a separate category.
        Returns train and test data.
    """
    
    # Identify columns for label encoding
    columns = list(train.columns[train.dtypes == 'object'])
       
    # Return data if no columns identified   
    if(len(columns) == 0):
        return train, test
       
    # Transform columns
    for column in columns: 
        
        # Filter data
        classes = train[column].unique()
        counts = train[column].value_counts()
        counts_classes = counts.index[counts <= threshold]
        
        # Set classes under threshold to 'identific'
        train.ix[train[column].isin(counts_classes), column] = 'identific'        
        test.ix[test[column].isin(counts_classes), column] = 'identific'          
        
        # Classes in test not in train sent to 'identific'
        test.ix[test[column].isin(classes) == False, column] = 'identific'
        
        # Perform label encoding
        le = LabelEncoder()
        le.fit(train[column])
        train[column] = le.transform(train[column]).astype(np.uint32)
        test[column] = le.transform(test[column]).astype(np.uint32)
        
    return train, test

In [ ]:
print "5. Encoding labels in data..."
train, test = encode_labels(train, test, threshold = 5)
print "Complete!"

### Remove data before Week 6 

In [ ]:
print "Shape Before (train): " + str(train.shape)
train = train[train['Semana'] > 5]  
train = train.reset_index(drop=True)
print "Shape After (train): " + str(train.shape)

### Reorder columns in data

In [ ]:
columns = [
            'Semana', 'Agencia_ID', 'Canal_ID', 'Ruta_SAK', 'Cliente_ID', 'Producto_ID'
            , 'Client_Type', 'short_product_name' 
            , 'weight', 'pieces', 'weight_per_piece'
            , 'Demanda_uni_equil', 'Demanda_uni_equil_tminus1', 'Demanda_uni_equil_tminus2'
            , 'Demanda_uni_equil_tminus3', 'Demanda_uni_equil_tminus4', 'Demanda_uni_equil_tminus5'
            , 'Agencia_ID_count', 'Canal_ID_count', 'Ruta_SAK_count', 'Cliente_ID_count'
            , 'Producto_ID_count', 'Client_Type_count'
          ]

train = train.reindex(columns = columns)
columns.remove('Demanda_uni_equil')
test = test.reindex(columns = columns) 

### Write data to CSV files

In [ ]:
check_memory_usage(train)
train.to_csv("train_modified.csv", index = False, header = True)
test.to_csv("test_modified.csv", index = False, header = True)

### Process complete!